In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Number of qubits
n = 2

# Pauli -> symplectic (x|z)
pauli_symp = {
    'I': (0, 0),
    'X': (1, 0),
    'Y': (1, 1),
    'Z': (0, 1)
}

# Hamiltonian: H = 1.0 * XX + 0.5 * ZZ + (-0.7) * IZ
H_terms = ['XX', 'ZZ', 'IZ']
H_coeffs = [1.0, 0.5, -0.7]

# Convert Pauli string to symplectic representation
def pauli_to_symplectic(pstr):
    x, z = [], []
    for p in pstr:
        xi, zi = pauli_symp[p]
        x.append(xi)
        z.append(zi)
    return np.array(x), np.array(z)

H_symplectic = [pauli_to_symplectic(p) for p in H_terms]

model = gp.Model("Best_Stabilizer")
model.Params.OutputFlag = 0  # silent

# Binary vars for stabilizer generators: 2 generators (for 2 qubits)
x = model.addVars(n, n, vtype=GRB.BINARY, name="x")
z = model.addVars(n, n, vtype=GRB.BINARY, name="z")

# Commutation constraints: [g_i, g_j] = 0
for i in range(n):
    for j in range(i+1, n):
        inner = []
        for k in range(n):
            a = model.addVar(vtype=GRB.BINARY)
            b = model.addVar(vtype=GRB.BINARY)
            model.addGenConstrAnd(a, [x[i, k], z[j, k]])
            model.addGenConstrAnd(b, [z[i, k], x[j, k]])
            inner.extend([a, b])
        total = model.addVar(vtype=GRB.INTEGER)
        model.addConstr(total == gp.quicksum(inner))
        is_even = model.addVar(vtype=GRB.BINARY)
        model.addConstr(total == 2 * is_even)

# Commutation with each Pauli term
m = len(H_terms)
commute = model.addVars(m, vtype=GRB.BINARY, name="commutes")

for j in range(m):
    pj_x, pj_z = H_symplectic[j]
    anticomm = []
    for i in range(n):
        terms = []
        for k in range(n):
            if pj_x[k]:
                a = model.addVar(vtype=GRB.BINARY)
                model.addGenConstrAnd(a, [z[i, k]])
                terms.append(a)
            if pj_z[k]:
                b = model.addVar(vtype=GRB.BINARY)
                model.addGenConstrAnd(b, [x[i, k]])
                terms.append(b)
        s = model.addVar(vtype=GRB.INTEGER)
        model.addConstr(s == gp.quicksum(terms))
        mod2 = model.addVar(vtype=GRB.BINARY)
        model.addConstr(s == 2 * (1 - mod2) + mod2)
        anticomm.append(mod2)
    # model.addConstr(commute[j] <= 1 - gp.quicksum(anticomm))
    for a in anticomm:
        model.addConstr(commute[j] <= 1 - a)


# Objective
model.setObjective(gp.quicksum(H_coeffs[j] * commute[j] for j in range(m)), GRB.MINIMIZE)
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Objective Value: {model.ObjVal}")
elif model.status == GRB.INFEASIBLE:
    print("Model infeasible")
    model.computeIIS()
    model.write("model.ilp")
else:
    print(f"Optimization ended with status {model.status}")


# Output
print("Best expectation value:", model.ObjVal)
print("Pauli terms that contribute:")
for j in range(m):
    if commute[j].X > 0.5:
        print(f"  {H_terms[j]} with coeff {H_coeffs[j]}")


Restricted license - for non-production use only - expires 2026-11-23
Objective Value: 0.0
Best expectation value: 0.0
Pauli terms that contribute:


In [2]:
import numpy as np

I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])

# Build the Hamiltonian
H = (
    1.0 * np.kron(X, X) +
    0.5 * np.kron(Z, Z) +
    -0.7 * np.kron(I, Z)
)

# Compute eigenvalues
eigvals = np.linalg.eigvalsh(H)
ground_energy = np.min(eigvals)
print("Ground state energy:", ground_energy)


Ground state energy: -1.7206555615733703
